In [83]:
import numpy as np
import pandas as pd
df=pd.read_csv('hongos.csv')
df = df.iloc[:,1:]
df_ordenado = df.sort_values(by='stalk-root', na_position='last')

x = df.drop(columns=['target'])


df_ordenado





,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,target
2821,x,f,g,t,n,f,c,b,p,t,...,g,p,p,w,o,p,n,y,d,e
2537,x,y,e,t,n,f,c,b,n,t,...,w,w,p,w,o,p,k,v,d,e
2538,x,f,e,t,n,f,c,b,n,t,...,w,g,p,w,o,p,k,v,d,e
2539,x,f,g,f,f,f,c,b,h,e,...,b,n,p,w,o,l,h,y,d,p
2540,x,f,p,f,c,f,w,n,n,e,...,w,w,p,w,o,p,n,s,d,p
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,k,s,n,f,n,a,c,b,y,e,...,o,o,p,o,o,p,b,c,l,e
8120,x,s,n,f,n,a,c,b,y,e,...,o,o,p,n,o,p,b,v,l,e
8121,f,s,n,f,n,a,c,b,n,e,...,o,o,p,o,o,p,b,c,l,e
8122,k,y,n,f,y,f,c,n,b,t,...,w,w,p,w,o,e,w,v,l,p


In [84]:
df_ordenado.describe()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,target
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,6,4,10,2,9,2,2,2,12,2,...,9,9,1,4,3,5,9,6,7,2
top,x,y,n,f,n,f,c,b,b,t,...,w,w,p,w,o,p,w,v,d,e
freq,3656,3244,2284,4748,3528,7914,6812,5612,1728,4608,...,4464,4384,8124,7924,7488,3968,2388,4040,3148,4208


In [85]:
df_ordenado.isnull().sum()

cap-shape                      0
cap-surface                    0
cap-color                      0
bruises                        0
odor                           0
gill-attachment                0
gill-spacing                   0
gill-size                      0
gill-color                     0
stalk-shape                    0
stalk-root                  2480
stalk-surface-above-ring       0
stalk-surface-below-ring       0
stalk-color-above-ring         0
stalk-color-below-ring         0
veil-type                      0
veil-color                     0
ring-number                    0
ring-type                      0
spore-print-color              0
population                     0
habitat                        0
target                         0
dtype: int64

In [86]:
df_missing = df_ordenado.copy()
mask = np.random.rand(*df_missing.iloc[:, :-1].shape) < 0.05
df_missing.iloc[:, :-1] = df_missing.iloc[:, :-1].mask(mask)
df_missing.isnull().sum()


cap-shape                    414
cap-surface                  395
cap-color                    384
bruises                      442
odor                         372
gill-attachment              408
gill-spacing                 371
gill-size                    379
gill-color                   410
stalk-shape                  429
stalk-root                  2761
stalk-surface-above-ring     414
stalk-surface-below-ring     450
stalk-color-above-ring       409
stalk-color-below-ring       399
veil-type                    397
veil-color                   400
ring-number                  429
ring-type                    415
spore-print-color            398
population                   381
habitat                      410
target                         0
dtype: int64

In [87]:
df2 = df_ordenado.iloc[:-3000].reset_index(drop=True)
df2.describe()


,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,target
count,5124,5124,5124,5124,5124,5124,5124,5124,5124,5124,...,5124,5124,5124,5124,5124,5124,5124,5124,5124,5124
unique,6,4,8,2,7,2,2,2,9,2,...,7,7,1,2,3,4,6,6,6,2
top,x,f,g,t,n,f,c,b,p,t,...,w,w,p,w,o,p,n,v,d,e
freq,2612,2034,1612,2923,2517,5106,4354,4494,1239,2626,...,2616,2568,5124,5116,4968,3222,1660,2122,2492,3037


In [88]:
df2.isnull().sum()

cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
target                      0
dtype: int64

In [89]:
from sklearn.preprocessing import LabelEncoder as Le
for col in df2.columns:
    encoder = Le()
    df2[col] = encoder.fit_transform(df2[col])

In [90]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_standard = scaler.fit_transform(df2.iloc[:, :-1])

df_X = pd.DataFrame(X_standard, columns=df2.columns[:-1], index=df2.index)

df_X.describe()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,5.124000e+03,5.124000e+03,5.124000e+03,5.124000e+03,5.124000e+03,5.124000e+03,5.124000e+03,5.124000e+03,5.124000e+03,5124.000000,...,5.124000e+03,5.124000e+03,5.124000e+03,5124.0,5.124000e+03,5.124000e+03,5.124000e+03,5.124000e+03,5.124000e+03,5.124000e+03
mean,-6.656138e-17,-1.664035e-17,-6.656138e-17,-1.774970e-16,4.437425e-17,8.320173e-17,4.437425e-17,-1.664035e-17,6.656138e-17,0.000000,...,2.218713e-17,-1.109356e-17,-1.109356e-17,0.0,2.773391e-18,-1.331228e-16,4.437425e-17,1.109356e-17,4.437425e-17,4.437425e-17
std,1.000098e+00,1.000098e+00,1.000098e+00,1.000098e+00,1.000098e+00,1.000098e+00,1.000098e+00,1.000098e+00,1.000098e+00,1.000098,...,1.000098e+00,1.000098e+00,1.000098e+00,0.0,1.000098e+00,1.000098e+00,1.000098e+00,1.000098e+00,1.000098e+00,1.000098e+00
min,-2.047344e+00,-1.184135e+00,-2.259379e+00,-1.152403e+00,-2.040295e+00,-1.684241e+01,-4.205340e-01,-3.744154e-01,-1.523129e+00,-1.025300,...,-2.429139e+00,-2.459571e+00,-2.450709e+00,0.0,-3.954392e-02,-5.850991e+00,-1.886356e+00,-1.082762e+00,-3.050365e+00,-7.345401e-01
25%,-8.537395e-01,-1.184135e+00,-6.486206e-01,-1.152403e+00,-8.865708e-01,5.937394e-02,-4.205340e-01,-3.744154e-01,-1.093412e+00,-1.025300,...,-9.045299e-01,-5.395767e-01,-5.282606e-01,0.0,-3.954392e-02,-9.437082e-02,-1.010365e+00,-1.082762e+00,-6.477102e-01,-7.345401e-01
50%,9.366675e-01,3.008156e-01,-1.117011e-01,8.677520e-01,8.440154e-01,5.937394e-02,-4.205340e-01,-3.744154e-01,1.957374e-01,0.975324,...,6.200791e-01,7.404192e-01,7.533717e-01,0.0,-3.954392e-02,-9.437082e-02,7.416180e-01,-1.702501e-01,1.531747e-01,-1.097420e-01
75%,9.366675e-01,1.043291e+00,9.621380e-01,8.677520e-01,8.440154e-01,5.937394e-02,-4.205340e-01,-3.744154e-01,1.055170e+00,0.975324,...,6.200791e-01,7.404192e-01,7.533717e-01,0.0,-3.954392e-02,-9.437082e-02,7.416180e-01,7.422620e-01,9.540596e-01,-1.097420e-01
max,9.366675e-01,1.043291e+00,1.499057e+00,8.677520e-01,1.420878e+00,5.937394e-02,2.377929e+00,2.670830e+00,1.914603e+00,0.975324,...,2.144688e+00,1.380417e+00,1.394188e+00,0.0,2.528834e+01,5.662249e+00,7.416180e-01,3.479798e+00,9.540596e-01,2.389450e+00


In [91]:
y = df2['target']

In [92]:

df_X.to_csv('hongos_datos.csv', index=False)

y.to_csv('hongos_target.csv', index=False, header=True)